# Resource Assignment Problem

### Problem statement

Consider three job positions: Tester, Java-Developer, and Architect.

Consider three resources: Carlos, Joe, and Monika.

Determine an assignment that ensures that each job is fulfilled and each resource is assigned to at most one job in order to maximize the total matching scores of the assignments.

### Data

The ability to perform each of the jobs by each of the resources is illustrated by the following matching scores table:

![Resource Allocation Problem Data Image](figs/rap_data.png)


**Assumption**: Only one resource can be assigned to a job, and at most one job can be assigned to a resource.

### Decision variables

The decision variable $x_{r,j} = 1$ represents that resource $r$ is assigned to job $j$, and 0 otherwise, for  $r=1,2,3$ and $j=1,2,3$.

### Constraints: jobs and resources

**Jobs constraints**

For each job $j=1,2,3$, exactly one resource from $r=1,2,3$ must be assigned.

Constraint (Tester=1): $x_{1,1} + x_{2,1} + x_{3,1} = 1$

Constraint (Java-Developer=2): $x_{1,2} + x_{2,2} + x_{3,2} = 1$

Constraint (Architect=3): $x_{1,3} + x_{2,3} + x_{3,3} = 1$

**Resources constraints**

For each resource = $r=1,2,3$, at most one job from $r=1,2,3$ can be assigned.

Constraint (Carlos=1): $x_{1,1} + x_{1,2} + x_{1,3}  \leq 1$

Constraint (Joe=2): $x_{2,1} + x_{2,2} + x_{2,3}  \leq 1$

Constraint (Monika=3): $x_{2,1} + x_{2,2} + x_{2,3}  \leq 1$

### Objective function

The objective function is to maximize the total matching score of the assignments while satisfying the jobs and resources constraints.

$$
\max \; 
(53x_{1,1} + 80x_{2,1} + 53x_{3,1}) 
+ (27x_{1,2} + 47x_{2,2} + 73x_{3,2})
+ (13x_{1,3} + 67x_{2,3} + 47x_{3,3})
$$

### Importing the Gurobi functions and classes

In [ ]:
# import gurobi library
import gurobipy as gp
from gurobipy import GRB

### Data

The list R contains the names of the three resources: Carlos, Joe, and Monika. 

The list J contains the names of the job positions: tester, java-developer, and architect

In [ ]:
# resources and jobs sets
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

$r \in R$ index and set of resources.

$j \in J$ index and set of Jobs.

The following “multidict” function describes the matching score associated with each possible combination of a resource and job

In [ ]:
# matching score data
combinations, ms = gp.multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})

### Creating the model

The following function generates an empty model object “model” and takes the string “RAP” model name as its argument.

In [ ]:
# Declare and initialize model
model = gp.Model("rap")

### Adding variables to the model

The decision variable $x_{r,j} = 1$ represents that resource $r$ is assigned to job $j$, and 0 otherwise, for $r \in R$ and $j \in J $.

The “addVars()” method defines the decision variables of the model object “model”.  

In [ ]:
# Create decision variables for the RAP model
x = model.addVars(combinations, vtype=GRB.BINARY, name="x")

### Adding constraints to the model: job and resource

**Job constraints**

For each job $j=1,2,3$, exactly one resource from $r=1,2,3$ must be assigned.

Constraint (Tester=1): $x_{1,1} + x_{2,1} + x_{3,1} = 1$

Constraint (Java-Developer=2): $x_{1,2} + x_{2,2} + x_{3,2} = 1$

Constraint (Architect=3): $x_{1,3} + x_{2,3} + x_{3,3} = 1$

The “addConstrs()” method defines the constraints of the model object “model”. 


In [ ]:
# create jobs  constraints
job = model.addConstrs((x.sum('*',j) == 1 for j in J), 'job')

$$
\sum_{r \: \in \: R} x_{r,j} = 1 \; \; \; \forall \; j \in J
$$

**Resource constraints**

For each resource $r=1,2,3$, at most one job from $r=1,2,3$ can be assigned.

Constraint (Carlos=1): $x_{1,1} + x_{1,2} + x_{1,3}  \leq 1$

Constraint (Joe=2): $x_{2,1} + x_{2,2} + x_{2,3}  \leq 1$

Constraint (Monika=3): $x_{3,1} + x_{3,2} + x_{3,3}  \leq 1$

The “addConstrs()” method defines the constraints of the model object “model”. 

In [ ]:
# create resources constraints
resource = model.addConstrs((x.sum(r,'*') <= 1 for r in R), 'resource')

$$
\sum_{j \: \in \: J} x_{r,j} \leq 1 \; \; \; \forall \; r \in R
$$

### Setting the objective function

The objective function is to maximize the total matching score of the assignments.

$$
\max \; (53x_{1,1} + 80x_{2,1} + 53x_{3,1})
 + (27x_{1,2} + 47x_{2,2} + 73x_{3,2})
 + (13x_{1,3} + 67x_{2,3} + 47x_{3,3})
$$

The “setObjective()” method defines the objective function of the model object “model”. 

In [ ]:
# The objective is to maximize total matching score of the assignments
model.setObjective(x.prod(ms), GRB.MAXIMIZE)

Notice that 
$$
(53x_{1,1} + 80x_{2,1} + 53x_{3,1}) = \sum_{r \; \in \; R} ms_{r,1}x_{r,1} \\
(27x_{1,2} + 47x_{2,2} + 73x_{3,2}) = \sum_{r \; \in \; R} ms_{r,2}x_{r,2} \\
(13x_{1,3} + 67x_{2,3} + 47x_{3,3})  = \sum_{r \; \in \; R} ms_{r,3}x_{r,3}
$$

Hence, the objective function can be expressed as follows

$$
\max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} ms_{r,j}x_{r,j}
$$

### Save model (.lp or .mps)

In [ ]:
# save model for inspection
model.write("rap.lp")
model.write("rap.mps")

### Optimizing the model

In [ ]:
# run optimization engine
model.optimize()

### Reporting the results

In [ ]:
def print_solution(model):
    for var in model.getVars():
        if abs(var.x) > 1e-6:
            print(f"{var.varName} = {var.x}")

    print(f"Total matching score: {model.objVal}")
    print(f"Runtime: {model.Runtime}")
    return None



In [ ]:
# display optimal values of decision variables
print_solution(model)   

## Refererences:

https://pypi.org/project/gurobipy/

https://support.gurobi.com/hc/en-us/categories/360000840931-Getting-Started-with-Gurobi

https://support.gurobi.com/hc/en-us/articles/360044290292-How-do-I-install-Gurobi-for-Python-

https://support.gurobi.com/hc/en-us/articles/17278438215313-Tutorial-Getting-Started-with-the-Gurobi-Python-API
